In [ ]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

from bofire_mixed_baselines.benchmarks.synthetic import DiscreteAckley
from bofire_mixed_baselines.data_models.strategies.api import (
    AnyStrategy,
    BARTGridStrategy,
    RelaxedSoboStrategy,
)
from bofire_mixed_baselines.data_models.strategies.mapper import strategy_map
from bofire_mixed_baselines.data_models.surrogates.api import BARTSurrogate

In [8]:
SMOKE_TEST = os.environ.get("SMOKE_TEST")
if SMOKE_TEST:
    num_runs = 1
    num_iters = 5
    verbose = False
else:
    num_runs = 2
    num_iters = 10
    verbose = True

In [9]:
benchmark = DiscreteAckley(discrete_dim=2, cont_dim=2)
domain = benchmark.domain
bart_strategy_dm = BARTGridStrategy(
    domain=domain,
    surrogate_specs=BARTSurrogate(
        inputs=domain.inputs,
        outputs=domain.outputs,
    ),
)
sobo_strategy_dm = RelaxedSoboStrategy(domain=benchmark.domain, seed=0)

strategy_data_models = [
    # bart_strategy_dm,
    sobo_strategy_dm,
]

In [10]:
def run_bo(strategy_dm: AnyStrategy, initial_experiments: pd.DataFrame) -> pd.DataFrame:
    strategy = strategy_map(strategy_dm)
    strategy.tell(initial_experiments)
    for _ in tqdm(range(num_iters), disable=not verbose):
        candidate = strategy.ask(1)
        y = benchmark.f(candidate, return_complete=True)
        strategy.tell(y)
    return strategy.experiments

In [ ]:
all_results = pd.DataFrame(
    columns=pd.MultiIndex.from_tuples([], names=("strategy", "run_idx"))
)
for strategy_dm in strategy_data_models:
    for run_idx in range(num_runs):
        samples = domain.inputs.sample(10, seed=run_idx)
        experiments = benchmark.f(samples, return_complete=True)
        results = run_bo(strategy_dm, experiments)
        y_lbl = domain.outputs.get_keys()[0]
        all_results[strategy_dm.type, run_idx] = results[y_lbl]

In [ ]:
import matplotlib.pyplot as plt


def plot_regret(ax: plt.Axes, experiments: pd.DataFrame):
    y_lbl = domain.outputs.get_keys()[0]
    opt = 0
    print(opt)
    model_results = (
        experiments.iloc[num_iters:].loc[:, ("RelaxedSoboStrategy",)].mean(axis=1)
    )
    regret = np.abs(model_results - opt)
    print(regret.shape)
    ax.plot(regret.cummin(), label="SOBO")


fig, ax = plt.subplots()
plot_regret(ax, all_results)

In [ ]:
all_results.iloc[num_iters:].loc[:, ("RelaxedSoboStrategy", slice(None))].mean(axis=1)